In [2]:
import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()

# These may be overridden in input file, but need not be specified there
periodsOnly=true; 
paths=[]; #unless overridden we will convert this to a list of straight paths
defaultPrecision=1e-30;
precision=[]; #if empty we will use the default precision for all systems

#load("~/git-projects/periods/integrate/data/current.sage")
load("~/git-projects/period-suite/current.sage")
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precision)==0:
    precision=[defaultPrecision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    if periodsOnly and j==steps:
        noOfdeqs=geometricGenus;
    else:
        noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "";
    print "Integrating system number", j;
    print "";
    %time transitionMatricies = [deqs[i-1].numerical_transition_matrix(paths[j-1], precision[j-1],assume_analytic=true) for i in [1..noOfdeqs]]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(300), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:geometricGenus]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIt took", t1-t0, "seconds in total.\n"
print periods


Integrating systems 1 through 5

Integrating system number 1

CPU times: user 10.5 s, sys: 156 ms, total: 10.6 s
Wall time: 10.6 s
Maximal error in transition matricies:
[3.9993473e-32, 3.9993473e-32, 4.8061152e-35, 1.6556212e-31, 1.5972817e-31, 1.4921127e-30, 1.1219174e-31, 4.8061152e-35, 1.6556212e-31, 1.5972817e-31, 1.4921127e-30, 1.1219174e-31, 1.5071348e-33, 2.6840740e-31, 1.5972817e-31, 1.4921127e-30, 1.1219174e-31, 1.5071348e-33, 2.6840740e-31, 8.5744197e-33, 8.5744197e-33]

Integrating system number 2

CPU times: user 746 ms, sys: 60.4 ms, total: 807 ms
Wall time: 762 ms
Maximal error in transition matricies:
[1.4789430e-44, 1.4789430e-44, 4.5522612e-44, 1.5925113e-45, 3.8565154e-41, 3.8852788e-41, 4.1612929e-41, 4.5522612e-44, 1.5925113e-45, 3.8565154e-41, 3.8852788e-41, 4.1612929e-41, 6.6482651e-42, 9.7438767e-45, 3.8565154e-41, 3.8852788e-41, 4.1612929e-41, 6.6482651e-42, 9.7438767e-45, 1.8342437e-41, 1.8342437e-41]

Integrating system number 3

CPU times: user 11 s, sys: 14

In [9]:
# Only for K3s
##
## Try a few values for "scale", do not exceed the negative exponent of accumulated error
##
scale = 23
##
##
periodVector=periods
u1 = periodVector.apply_map(real)[0].list()
u2 = periodVector.apply_map(imag)[0].list()
M = matrix([u1,u2])
#M = matrix([u1,u2])
#proj = (M.pseudoinverse() * M)
proj = M.transpose()
lattice = block_matrix([[(10^scale*proj).apply_map(lambda x : x.round()), matrix.identity(proj.dimensions()[0])]])
generatorNorms = [r.norm().n() for r in lattice.LLL().rows()]
consecutiveRatios = [generatorNorms[i]/generatorNorms[i-1] for i in [1..20]]

j=0; rank=1;
while j <= 19 :
    if consecutiveRatios[j] > 100 :
        j=20
    rank = rank + 1
    j=j+1
    
if generatorNorms[0] > 100 :
    rank = 1

rank

1

In [35]:
generatorNorms

[1.41421356237310,
 1.73205080756888,
 2.00000000000000,
 2.00000000000000,
 2.23606797749979,
 2.44948974278318,
 2.44948974278318,
 2.64575131106459,
 2.64575131106459,
 2.44948974278318,
 2.64575131106459,
 3.00000000000000,
 3.16227766016838,
 43880.3838064345,
 45679.9549036555,
 51086.7701269125,
 59214.2089282632,
 57768.7252412584,
 58580.7529654579,
 75789.3881028208,
 75501.3145117355]